# 📚 Project Based Learning (PBL)
## Integrasi Big Data, Blockchain, AI, dan Machine Learning dalam Kajian Sastra Indonesia

Notebook ini mencakup:
- Ekstraksi teks dari gambar menggunakan OCR
- Normalisasi ejaan lama ke ejaan baru
- Klasifikasi genre sastra menggunakan Machine Learning
- Simulasi Blockchain untuk distribusi karya sastra
- Visualisasi data sastra Indonesia


# 🚀 Proyek Analisis dan Registrasi Karya Sastra Digital

Selamat datang di Notebook Analisis Karya Sastra!

**Tujuan Proyek:**
Notebook ini mendemonstrasikan alur kerja lengkap untuk mengubah sebuah karya sastra dari format gambar menjadi data digital yang terstruktur, teranalisis, dan tercatat.

**Alur Kerja:**
1.  **Instalasi Prasyarat:** Panduan instalasi Tesseract OCR dan pustaka Python.
2.  **Ekstraksi Teks (OCR):** Mengambil teks dari file gambar (`.jpg`, `.png`).
3.  **Normalisasi Teks:** Membersihkan dan memperbaiki teks, termasuk mengubah ejaan lama.
4.  **Analisis Genre (AI):** Menggunakan model *Machine Learning* untuk memprediksi genre sastra (Puisi, Prosa, Drama).
5.  **Registrasi Blockchain:** Mensimulasikan pencatatan karya sastra ke dalam *blockchain* sebagai bukti digital yang tidak dapat diubah.
6.  **Visualisasi Data:** Menampilkan grafik distribusi genre dari data yang digunakan untuk melatih model AI.

## Langkah 0: Instalasi dan Konfigurasi Prasyarat

Sebelum menjalankan kode, ada beberapa hal yang harus diinstal terlebih dahulu.

### Bagian A: Instalasi Tesseract OCR (Untuk Windows)

Tesseract adalah "mesin" yang akan membaca teks dari gambar.

1.  **Unduh Installer:** Buka link berikut dari UB Mannheim yang menyediakan installer Tesseract untuk Windows:
    * [https://github.com/UB-Mannheim/tesseract/wiki](https://github.com/UB-Mannheim/tesseract/wiki)
2.  **Pilih Versi:** Unduh versi installer yang direkomendasikan (misalnya, `tesseract-ocr-w64-setup-v5.x.x...exe`).
3.  **Jalankan Installer:**
    * Saat instalasi berjalan, Anda akan sampai pada tahap "Choose Components". **PENTING:** Pastikan Anda mencentang "Additional language data" untuk mengikutsertakan direktori `tessdata`.
    * **CATAT LOKASI INSTALASI!** Perhatikan di mana Anda menginstal Tesseract. Lokasi default-nya biasanya `C:\Program Files\Tesseract-OCR`. Path ini akan kita butuhkan di dalam kode Python.

### Bagian B: Menambahkan Paket Bahasa Indonesia

Agar Tesseract bisa mengenali teks berbahasa Indonesia dengan baik, kita perlu menambahkan file bahasa.

1.  **Unduh File Bahasa:** Buka link ke repositori data Tesseract resmi berikut:
    * Langsung unduh file `ind.traineddata` dari sini: [**Download Indonesian Language Data**](https://github.com/tesseract-ocr/tessdata/raw/main/ind.traineddata)
2.  **Pindahkan File:**
    * Salin file `ind.traineddata` yang baru saja Anda unduh.
    * Tempel file tersebut ke dalam folder `tessdata` di lokasi instalasi Tesseract Anda.
    * Contoh path: `C:\Program Files\Tesseract-OCR\tessdata\`

### Bagian C: Instalasi Pustaka Python

Jalankan sel kode di bawah ini untuk menginstal semua pustaka Python yang dibutuhkan oleh proyek ini.

In [35]:
# Jalankan sel ini untuk menginstal semua library yang dibutuhkan
!pip install jupyter pillow matplotlib scikit-learn pytesseract


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Langkah 1: Import Pustaka dan Konfigurasi Awal

Sekarang kita akan mengimpor semua pustaka yang sudah diinstal dan mengatur path ke Tesseract.

In [44]:
import os
import time
import hashlib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from collections import Counter
import pytesseract
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

def konfigurasi_tesseract():
    """Mengatur path untuk Tesseract OCR. Sesuaikan path di bawah ini."""
    try:
        # ‼️ PENTING: Sesuaikan path ini dengan lokasi instalasi Tesseract di komputer Anda!
        tesseract_path = r"D:\SOFTWARE\Tesseract\tesseract.exe"
        pytesseract.pytesseract.tesseract_cmd = tesseract_path
        # Cek apakah Tesseract terdeteksi
        print(f"✅ Tesseract berhasil dikonfigurasi. Versi: {pytesseract.get_tesseract_version()}")
        return True
    except Exception as e:
        print("❌ GAGAL: Tesseract tidak ditemukan atau path salah.")
        print("   Pastikan Tesseract sudah terinstal dan path di atas sudah benar.")
        print(f"   Detail Error: {e}")
        return False

# Jalankan konfigurasi
TESSERACT_READY = konfigurasi_tesseract()

✅ Tesseract berhasil dikonfigurasi. Versi: 5.5.0.20241111


## Langkah 2: Simulasi Gambar dan Ekstraksi Teks (OCR)

Untuk memastikan Notebook ini bisa langsung berjalan, kita akan membuat sebuah file gambar contoh (`halaman_buku.jpg`) secara otomatis. Kemudian, kita akan mengekstrak teks dari gambar tersebut dan melakukan normalisasi ejaan.

In [45]:
def buat_gambar_contoh(nama_file="halaman_buku.jpg"):
    """Membuat file gambar JPEG dengan teks contoh."""
    print(f"Membuat gambar contoh '{nama_file}'...")
    text_to_draw = (
        "Sebuah tjerita dari masa laloe.\n\n"
        "Langit soenji memandangkoe.\n"
        "Mengapa engkau pergi, tanja sang pangeran.\n"
        "Djalan ini masih pandjang."
    )
    
    # Buat gambar putih
    img = Image.new('RGB', (600, 400), color = 'white')
    d = ImageDraw.Draw(img)
    
    # Gunakan font default atau coba cari font lain
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()
        
    d.text((30,30), text_to_draw, fill='black', font=font)
    img.save(nama_file)
    print(f"✅ Gambar contoh '{nama_file}' berhasil dibuat.")
    return nama_file


def lakukan_ocr_dan_normalisasi(path_gambar):
    """Membaca teks dari gambar dan menormalkan ejaan lama."""
    print(f"\n📖 Membaca Teks dari Gambar: '{path_gambar}'")
    try:
        img = Image.open(path_gambar)
        teks_ocr = pytesseract.image_to_string(img, lang='ind')
        
        if not teks_ocr.strip():
            print("   ⚠️ Peringatan: OCR tidak menghasilkan teks.")
            return None
            
        print("   ✅ Teks mentah berhasil diekstrak.")
        
        # Normalisasi ejaan lama
        mapping_ejaan = {'dj': 'j', 'tj': 'c', 'oe': 'u', 'nj': 'ny', 'sj': 'sy', 'ch': 'kh'}
        teks_normal = teks_ocr
        for lama, baru in mapping_ejaan.items():
            teks_normal = teks_normal.replace(lama, baru)
        
        print("   ✅ Ejaan lama telah dinormalisasi.")
        return teks_normal
        
    except FileNotFoundError:
        print(f"   ❌ GAGAL: File gambar '{path_gambar}' tidak ditemukan.")
        return None
    except Exception as e:
        print(f"   ❌ GAGAL: Terjadi error saat proses OCR. Detail: {e}")
        return None

# Buat gambar dan proses
nama_gambar = buat_gambar_contoh()
teks_karya = lakukan_ocr_dan_normalisasi(nama_gambar)

# Tampilkan hasilnya
if teks_karya:
    print("-" * 50)
    print(">>> Teks Final Hasil Normalisasi:")
    print(teks_karya)
    print("-" * 50)

Membuat gambar contoh 'halaman_buku.jpg'...
✅ Gambar contoh 'halaman_buku.jpg' berhasil dibuat.

📖 Membaca Teks dari Gambar: 'halaman_buku.jpg'
   ✅ Teks mentah berhasil diekstrak.
   ✅ Ejaan lama telah dinormalisasi.
--------------------------------------------------
>>> Teks Final Hasil Normalisasi:
Sebuah cerita dari masa lalu.

Langit sunyi memandangku.
Mengapa engkau pergi, tanya sang pangeran.
Djalan ini masih panyang.

--------------------------------------------------


## Langkah 3: Membangun Model AI untuk Klasifikasi Genre

Di sini, kita mendefinisikan dan melatih model *Machine Learning* menggunakan `scikit-learn`. Model ini akan belajar dari `corpus_teks` untuk mengenali pola bahasa dari setiap genre sastra.

In [50]:
def latih_model_genre():
    """Membangun dan melatih model AI untuk klasifikasi genre sastra."""
    print("\n🧠 Membangun dan Melatih Model AI Klasifikasi Genre...")
    
    corpus_teks = [
        "Aku adalah api yang membakar jiwamu...", "Langit menangis bersama rinduku yang lara...",
        "Sajak sunyi di balik kelamnya malam...", "Bayangmu menari di antara gugusan bintang...",
        "Di bawah rembulan, kutulis syair kerinduan...", "Kata-kata menjelma embun di ujung daun...",
        "Dialog antara dua insan di balik tirai waktu...", "Mengapa kau tinggalkan aku, tanya si wanita tua dengan lirih.",
        "Kita tak bisa kembali ke masa lalu! (sambil menatap nanar)", "Kau berdusta! teriaknya sambil memukul meja dengan keras.",
        "Jangan sentuh aku! Pergi dari sini sekarang juga!", "(Terdiam sejenak) Aku... aku tidak tahu harus berbuat apa.",
        "Ia berjalan menuju pasar sambil membawa secarik kertas berisi daftar belanja.", "Pagi itu hujan turun perlahan, membasahi jalanan yang lengang dan sepi.",
        "Dengan tangan gemetar, ia membuka kembali buku harian lamanya.", "Sebuah surat cinta yang telah menguning ditemukan di bawah lantai kayu.",
        "Cerita berawal dari sebuah desa kecil di kaki gunung yang subur.", "Matahari terbenam di ufuk barat, melukiskan warna jingga di langit."
    ]
    label_genre = [
        'Puisi', 'Puisi', 'Puisi', 'Puisi', 'Puisi', 'Puisi',
        'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama',
        'Prosa', 'Prosa', 'Prosa', 'Prosa', 'Prosa', 'Prosa'
    ]
    
    stop_words_indonesia = ['yang', 'di', 'dan', 'ke', 'ini', 'itu', 'dengan', 'adalah', 'ia', 'si', 'sambil', 'telah', 'aku', 'kau']

    model = Pipeline([
        ('tfidf', TfidfVectorizer(ngram_range=(1, 2), stop_words=stop_words_indonesia)),
        ('clf', MultinomialNB(alpha=0.1))
    ])
    
    model.fit(corpus_teks, label_genre)
    print("   ✅ Model AI siap digunakan.")
    return model, label_genre

def prediksi_genre(model, teks):
    """Memprediksi genre dari sebuah teks dan menampilkan probabilitasnya."""
    print("\n🤖 Menganalisis Teks Menggunakan Model AI...")
    prediksi = model.predict([teks])[0]
    probabilitas = model.predict_proba([teks])[0]
    
    classes = model.classes_
    skor = {genre: f"{p*100:.2f}%" for genre, p in zip(classes, probabilitas)}
    
    print(f"   Prediksi Genre    : {prediksi}")
    print(f"   Tingkat Keyakinan : {max(probabilitas)*100:.2f}%")
    print(f"   Skor Probabilitas : {skor}")
    return prediksi

# Latih model
model_ai, labels_pelatihan = latih_model_genre()

# Lakukan prediksi pada teks hasil OCR
if teks_karya:
    genre_prediksi = prediksi_genre(model_ai, teks_karya)


🧠 Membangun dan Melatih Model AI Klasifikasi Genre...
   ✅ Model AI siap digunakan.

🤖 Menganalisis Teks Menggunakan Model AI...
   Prediksi Genre    : Drama
   Tingkat Keyakinan : 71.13%
   Skor Probabilitas : {'Drama': '71.13%', 'Prosa': '15.55%', 'Puisi': '13.32%'}


## Langkah 4: Simulasi Blockchain untuk Registrasi Karya

Setelah karya dianalisis, kita bisa mencatatnya ke dalam sebuah sistem yang aman dan tidak dapat diubah. Blockchain adalah teknologi yang cocok untuk ini. Di sini, kita membuat simulasi sederhana dari proses tersebut.

In [51]:
class Block:
    """Mendefinisikan struktur satu blok dalam Blockchain."""
    def __init__(self, index, data, prev_hash):
        self.index = index
        self.timestamp = time.time()
        self.data = data
        self.prev_hash = prev_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.index}{self.timestamp}{str(self.data)}{self.prev_hash}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    def __repr__(self):
        """Representasi string yang rapi untuk sebuah blok."""
        return (
            f"   | ----------------- Block #{self.index} ----------------- |\n"
            f"   | Timestamp   : {time.ctime(self.timestamp)}\n"
            f"   | Data        : {self.data}\n"
            f"   | Hash        : {self.hash[:40]}...\n"
            f"   | Prev. Hash  : {self.prev_hash[:40]}...\n"
            f"   | ----------------------------------------------- |"
        )

class Blockchain:
    """Mengelola rantai blok."""
    def __init__(self):
        self.chain = [Block(0, "Genesis Block: Rantai Dimulai", "0")]

    def tambah_blok(self, data):
        prev_hash = self.chain[-1].hash
        new_block = Block(len(self.chain), data, prev_hash)
        self.chain.append(new_block)
        print("\n   ✅ Blok baru berhasil ditambahkan ke rantai.")

def tampilkan_blockchain(bc):
    """Menampilkan seluruh isi blockchain dengan format yang baik."""
    print("\n🔗 Menampilkan Catatan di Buku Besar Digital (Blockchain)...")
    for block in bc.chain:
        print(block)
        if block.index < len(bc.chain) - 1:
            print("                   V (terhubung ke)")

# Inisialisasi Blockchain dan tambahkan karya baru jika ada
if teks_karya:
    blockchain_sastra = Blockchain()
    
    data_karya_baru = {
        "judul": f"Karya dari {nama_gambar}",
        "penulis": "Anonim (via OCR)",
        "teks_singkat": f"'{teks_karya[:50].strip()}...'",
        "genre_terdeteksi": genre_prediksi,
        "status": "Terdaftar Digital"
    }
    blockchain_sastra.tambah_blok(data_karya_baru)
    tampilkan_blockchain(blockchain_sastra)


   ✅ Blok baru berhasil ditambahkan ke rantai.

🔗 Menampilkan Catatan di Buku Besar Digital (Blockchain)...
   | ----------------- Block #0 ----------------- |
   | Timestamp   : Sat Jun  7 16:32:06 2025
   | Data        : Genesis Block: Rantai Dimulai
   | Hash        : a496fd8eb2c6286e4c8e361b046e62e7809a93b3...
   | Prev. Hash  : 0...
   | ----------------------------------------------- |
                   V (terhubung ke)
   | ----------------- Block #1 ----------------- |
   | Timestamp   : Sat Jun  7 16:32:06 2025
   | Data        : {'judul': 'Karya dari halaman_buku.jpg', 'penulis': 'Anonim (via OCR)', 'teks_singkat': "'Sebuah cerita dari masa lalu.\n\nLangit sunyi memand...'", 'genre_terdeteksi': 'Drama', 'status': 'Terdaftar Digital'}
   | Hash        : c50a2966b7f0ea4c00a18701321f3ef78cbf038b...
   | Prev. Hash  : a496fd8eb2c6286e4c8e361b046e62e7809a93b3...
   | ----------------------------------------------- |


## Langkah 5: Visualisasi Data

Sebagai langkah terakhir, kita akan membuat visualisasi data untuk melihat komposisi genre dalam dataset yang kita gunakan untuk melatih model AI.

In [53]:
import matplotlib.ticker as mtick

def visualisasikan_hasil_prediksi(classes, probabilities, teks_asli):
    """
    Membuat grafik batang horizontal untuk menunjukkan skor kepercayaan (probabilitas)
    hasil prediksi model AI untuk setiap genre.
    """
    print("\n📊 Membuat Visualisasi Hasil Analisis AI...")
    
    # Mengambil potongan teks untuk judul
    teks_singkat = (teks_asli[:75] + '...') if len(teks_asli) > 75 else teks_asli
    
    # Menentukan warna: bar tertinggi akan disorot
    colors = ['skyblue'] * len(classes)
    max_prob_index = np.argmax(probabilities)
    colors[max_prob_index] = 'salmon'

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 7))

    # Membuat bar chart horizontal
    bars = ax.barh(classes, probabilities, color=colors)
    
    # Menambahkan label persentase di ujung setiap bar
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax.text(width + 0.01, bar.get_y() + bar.get_height()/2, f'{width:.1%}', 
                va='center', ha='left', fontsize=12)

    ax.set_title(f"Analisis Keyakinan Model untuk Teks:\n'{teks_singkat.replace(chr(10), ' ')}'", fontsize=16, pad=20)
    ax.set_xlabel('Tingkat Keyakinan (Probabilitas)', fontsize=12)
    ax.set_ylabel('Genre Sastra', fontsize=12)
    
    # Format sumbu X sebagai persentase
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    ax.set_xlim(0, 1.1) # Beri sedikit ruang ekstra di kanan
    ax.tick_params(axis='y', labelsize=12)
    
    # Balik urutan agar genre dengan probabilitas tertinggi ada di atas
    ax.invert_yaxis()

    print("   ✅ Grafik analisis keyakinan model akan ditampilkan di bawah ini.")
    plt.tight_layout()
    plt.show()